Author: Suhan Shetty (suhan.n.shetty@gmail.com | suhan.shetty@idiap.ch)

This notebook implements multilinear PCA based regression for tensor time series modeling

References:

 - Dynamic tensor time series modeling and analysis, https://ieeexplore.ieee.org/abstract/document/7798500

 - MULTILINEAR TENSOR REGRESSION FOR LONGITUDINAL RELATIONAL DATA:
 https://www.jstor.org/stable/43826417
 
 - Multilinear Dynamical Systems for Tensor Time Series: https://people.eecs.berkeley.edu/~russell/papers/nips13-tensor.pdf


In [46]:
import numpy as np
import tensorly as tl
from tensorly.tenalg import multi_mode_dot
from tensorly.base import unfold
from scipy.sparse.linalg import eigsh
from scipy.linalg import pinv
from tensorly.tenalg import kronecker as kron


In [47]:
N = 3 # dimension of each output tensor Yi's
T = 10 # length of time series
shape_Yi = [20,20,20]
I = [T,*shape_Yi]

# Generate data Y from some low-rank structure 
J0 = [10,10,10] # ranks of the factor matrices
U0 = [np.random.rand(I[n+1],J0[n]) for n in range(N)] #Factors
X0 = np.random.rand(T,*J0) # Core array

Y = multi_mode_dot(X0,U0, modes=[n+1 for n in range(N)]) #Output tensor
    

In [48]:
Y_mean = np.average(Y,axis=0)
Yn = Y - Y_mean # mean zero

In [60]:
def mpca(Y,J,tol=1e-2):
    '''
    Y: an N-dimensional Tensor
    J: Desired ranks of the tucker factors (increase J for better approximation)
    tol: tolerance for convergence of mpca iterations
    '''
    Yn = Y-np.average(Y,axis=0)
    # Initialization
    U = [None]*(len(Yn.shape)-1)
    for n in range(N): # across all modes
        mode_n_sum = 0
        for t in range(T):
            unfold_along_n = unfold(Yn[t],mode=n)
            mode_n_sum += unfold_along_n@unfold_along_n.T
        U[n] = eigsh(mode_n_sum,k=J[n])[1] # faster way to compute eigen vectors (for symmetric matrices)
        # Note: The most significant eigen vectors are arranged in the last columns of U
    
    # Local optimization
    X = multi_mode_dot(Yn,U,modes=[1+i for i in range(N)], transpose=True)
    Yn_apprx0 = multi_mode_dot(X,U,modes=[1+i for i in range(N)])
    convergence = False
    for m in range(500): #iterate until convergence
        for n in range(N):
            U_ = U[:]
            U_.pop(n)
            mode_n_sum = 0
            U_kron = kron(U_,reverse=True)
            for t in range(T):
                unfold_along_n = unfold(Yn[t],mode=n)
                mode_n_sum += unfold_along_n@U_kron@U_kron.T@unfold_along_n.T
            U[n] = eigsh(mode_n_sum,k=J[n])[1]
        
        X = multi_mode_dot(Yn,U,modes=[1+i for i in range(N)], transpose=True) # Current cores
        
        # Check convergence
        Yn_apprx = multi_mode_dot(X,U,modes=[1+i for i in range(N)])#Curent approximation
        if np.linalg.norm(Yn_apprx-Yn_apprx0) < tol:
            print("mpca converged at iteration: ",m)
            convergence = True
            break
        else:
            Yn_apprx0 = 1*Yn_apprx
    
    if convergence==False:
        print("mpca has not yet converged")
        
    X = multi_mode_dot(Y,U,modes=[1+i for i in range(N)], transpose=True)
    Y_apprx = multi_mode_dot(X,U,modes=[1+i for i in range(N)])
    print("Error in approximation: ", np.linalg.norm(Y-Y_apprx)/np.linalg.norm(Y))

    return X,U

In [61]:
X,U = mpca(Y, J=[10,10,10]) 

mpca converged at iteration:  0
Error in approximation:  9.534671749413086e-16


In [209]:
# Multilinear Regression
# Ref:MULTILINEAR TENSOR REGRESSION FOR LONGITUDINAL RELATIONAL DATA:
# https://www.jstor.org/stable/43826417?seq=1#metadata_info_tab_contents
def multilinear_regression(X,Y,tol=0.0001):
    '''
    X: Tx?x?x..x? : {X_i}, i=1,..,T
    Y: Tx?x?x..x?: {Y_i}, i=1,...,T 
    N: dimension of the tensor X_i (or Y_i)
    J: ranks of the factor matrices
    return the model: Y_i = X_ix{U_1,..,U_N}
    '''
    I = Y.shape
    J = X.shape
    U = [100*np.random.rand(I[n+1],J[n+1]) for n in range(N)] 
    #Note: Fix the initialization. Somehow, smaller values of initialization, results in slower convergence.
    # Understand this problem
    U.insert(0,np.eye(I[0])) 
    Y_apprx0 = multi_mode_dot(X,U,modes=[i for i in range(N+1)])
    convergence = False
    for m in range(1000):
        for n in range(N):
            U_ = U[:]
            U_[n+1] = np.eye(J[n+1])
            X_tmp = multi_mode_dot(X,U_,modes=[i for i in range(N+1)]) 
            X_tmp_n = unfold(X_tmp,mode=n+1)
            Y_n = unfold(Y,mode=n+1)
            U[n+1] = Y_n@pinv(X_tmp_n)
            if n<(N-1):#to make sure the values dont blow too much
                U[n+1] = U[n+1]/np.linalg.norm(U[n+1])
       
        # Check convergence
        Y_apprx = multi_mode_dot(X,U,modes=[i for i in range(N+1)])#Curent approximation
        
        if np.linalg.norm(Y_apprx-Y_apprx0) < tol:
            print("Regression converged at iteration: ",m)
            convergence = True
            break
        else:
            if (m+1)%1==0:
                print("Current error in approximation:", np.linalg.norm(Y-Y_apprx)/np.linalg.norm(Y))
            Y_apprx0 = 1*Y_apprx

    
    if convergence==False:
        print("Multilinear regression has not yet converged")
    
    print("Final error in approximation:", np.linalg.norm(Y-Y_apprx)/np.linalg.norm(Y))
    
    return U[1:]


In [210]:
U = multilinear_regression(X,Y,tol=0.0001)

Current error in approximation: 0.035951007244001146
Current error in approximation: 0.009199243944314563
Current error in approximation: 0.009169279837492952
Current error in approximation: 0.009167521700683723
Current error in approximation: 0.00916598789245401
Current error in approximation: 0.009164489123802147
Current error in approximation: 0.00916301268049994
Current error in approximation: 0.009161556810182925
Current error in approximation: 0.009160120761988808
Current error in approximation: 0.00915870392377438
Current error in approximation: 0.009157305726025819
Current error in approximation: 0.009155925625940853
Current error in approximation: 0.009154563103158308
Current error in approximation: 0.00915321765790729
Current error in approximation: 0.009151888809799731
Current error in approximation: 0.009150576096833361
Current error in approximation: 0.0091492790744874
Current error in approximation: 0.009147997314876505
Current error in approximation: 0.009146730405952286

Current error in approximation: 0.009006185258728044
Current error in approximation: 0.00900540896412059
Current error in approximation: 0.009004630790699944
Current error in approximation: 0.009003850733017045
Current error in approximation: 0.009003068786929847
Current error in approximation: 0.009002284949619282
Current error in approximation: 0.009001499219604584
Current error in approximation: 0.009000711596757908
Current error in approximation: 0.008999922082318002
Current error in approximation: 0.00899913067890312
Current error in approximation: 0.00899833739052305
Current error in approximation: 0.008997542222590176
Current error in approximation: 0.008996745181929557
Current error in approximation: 0.008995946276787884
Current error in approximation: 0.0089951455168414
Current error in approximation: 0.008994342913202719
Current error in approximation: 0.008993538478426382
Current error in approximation: 0.008992732226513263
Current error in approximation: 0.00899192417291361

Current error in approximation: 0.008872797308993997
Current error in approximation: 0.008872206844290504
Current error in approximation: 0.00887161940349679
Current error in approximation: 0.008871034977095679
Current error in approximation: 0.008870453555317945
Current error in approximation: 0.00886987512815183
Current error in approximation: 0.008869299685352234
Current error in approximation: 0.008868727216449818
Current error in approximation: 0.00886815771075994
Current error in approximation: 0.008867591157391313
Current error in approximation: 0.008867027545254518
Current error in approximation: 0.00886646686307036
Current error in approximation: 0.00886590909937801
Current error in approximation: 0.008865354242542922
Current error in approximation: 0.008864802280764689
Current error in approximation: 0.008864253202084534
Current error in approximation: 0.008863706994392843
Current error in approximation: 0.008863163645436342
Current error in approximation: 0.00886262314282513

Current error in approximation: 0.008799124818405729
Current error in approximation: 0.008798901278165714
Current error in approximation: 0.008798678679395773
Current error in approximation: 0.008798457016465673
Current error in approximation: 0.008798236283786077
Current error in approximation: 0.008798016475808225
Current error in approximation: 0.008797797587023579
Current error in approximation: 0.008797579611963605
Current error in approximation: 0.008797362545199411
Current error in approximation: 0.008797146381341405
Current error in approximation: 0.008796931115039031
Current error in approximation: 0.00879671674098054
Current error in approximation: 0.008796503253892526
Current error in approximation: 0.008796290648539795
Current error in approximation: 0.00879607891972494
Current error in approximation: 0.008795868062288117
Current error in approximation: 0.008795658071106778
Current error in approximation: 0.008795448941095286
Current error in approximation: 0.00879524066720

Current error in approximation: 0.008768757658956027
Current error in approximation: 0.008768645332196031
Current error in approximation: 0.008768533350267867
Current error in approximation: 0.008768421711600223
Current error in approximation: 0.008768310414631155
Current error in approximation: 0.008768199457807917
Current error in approximation: 0.008768088839587029
Current error in approximation: 0.008767978558434116
Current error in approximation: 0.008767868612823857
Current error in approximation: 0.00876775900123997
Current error in approximation: 0.00876764972217507
Current error in approximation: 0.008767540774130672
Current error in approximation: 0.008767432155617096
Current error in approximation: 0.008767323865153415
Current error in approximation: 0.008767215901267356
Current error in approximation: 0.008767108262495316
Current error in approximation: 0.008767000947382244
Current error in approximation: 0.008766893954481563
Current error in approximation: 0.00876678728235

Current error in approximation: 0.008752906573852054
Current error in approximation: 0.008752837735585772
Current error in approximation: 0.008752769063160958
Current error in approximation: 0.008752700555977632
Current error in approximation: 0.008752632213438713
Current error in approximation: 0.008752564034949917
Current error in approximation: 0.008752496019919856
Current error in approximation: 0.008752428167759896
Current error in approximation: 0.008752360477884235
Current error in approximation: 0.008752292949709875
Current error in approximation: 0.00875222558265661
Current error in approximation: 0.008752158376146906
Current error in approximation: 0.008752091329606077
Current error in approximation: 0.008752024442462059
Current error in approximation: 0.008751957714145577
Current error in approximation: 0.008751891144089944
Current error in approximation: 0.008751824731731288
Current error in approximation: 0.008751758476508302
Current error in approximation: 0.0087516923778

In [147]:
Y.shape

(10, 20, 20, 20)

In [154]:
# Compute Array-normal distribution 

# Compute Matrix root--------------------------------------------------------------------------------------
# Compute A's from Cov: Cov = U*S*Vh, A = U*Sqrt(S)
def mat_root(Cov,tol=1e-9):
    U,s,Vh = np.linalg.svd(Cov)
    idx = s>tol
    s = s[idx]
    U = U[:,idx]   
    s_r = np.sqrt(s)
    S_r = np.diag(s_r)
    S_inv_r = np.diag(1/s_r)
    A = U@S_r
    A_inv = S_inv_r@U.T
    return (A, A_inv) # retun both the sqrt and its inverse

# Separable Covariance estimation for N-way array data using MLE
def array_normal(X, tol=1e-2): 
    '''
    Data points {Xi}, i=1,..,T are arranged into one array X
    Input tensor X. Xi = X[i], i =1,..,T are the data points
    '''    
    I = X.shape 
    T = X.shape[0]
    N  = len(I)-1 # dimension of the data points X_i
    
    # Compute the mean:
    Xavg = np.average(X,axis=0)
    
    Xe = X - Xavg

    A = [100*np.random.rand(I[i+1],I[i+1]) for i in range(N)]
    Cov = [A_@A_.T for A_ in A]   
    
    # X[i] ~ Xavg + Z x {Cov1, Cov2,..., CovN}, and CovK = Ak*Ak'
    A_inv = [None]*T
    for n in range(N):
        (A[n],A_inv[n]) = mat_root(Cov[n])
    A_inv.insert(0,np.identity(T))
    A.insert(0,np.identity(T))
    Cov.insert(0,np.identity(T))
    
    for reps in range(100):
        for n in range(N): #iterate over each mode
            A_inv_ = A_inv[:]
            A_ = A[:]
            Cov_ = Cov[:]
            A_[n+1] = np.eye(I[n+1])
            A_inv_[n+1] = np.eye(I[n+1])
            Xe_tmp = multi_mode_dot(Xe,A_inv_,modes=[i for i in range(N+1)])
            Xe_tmp_n = unfold(Xe_tmp,mode=n+1)
            c = T*np.prod(I[1:])/I[n+1]
            Cov[n+1] = (Xe_tmp_n@Xe_tmp_n.T)/c
            if n<(N-1):#to make sure the values dont blow too much
                Cov[n+1] = Cov[n+1]/np.linalg.norm(Cov[n+1])
                
            (A[n+1],A_inv[n+1]) = mat_root(Cov[n+1])
         
        err = np.linalg.norm(kron(Cov[1:])-kron(Cov_[1:]))
        if err<tol:
            print("MLE for array normal converged in ", reps, " steps")
            break
            
    return (Xavg,Cov[1:],A[1:], A_inv[1:])

# Sampling from array-normal distribution---------------------------------------------------------------------------
def anormal_sampling(Xavg,Cov):
    N = len(Xavg.shape)
    # Compute the matrix-square-root (Cov = A*A') of covariance matrices
    A = [mat_root(Cov[n])[0] for n in range(N)]
    Z = np.random.randn(*Xavg.shape)
    X = Xavg + multi_mode_dot(Z,A,modes=[n for n in range(N)])
    return X


In [155]:
X0 = np.random.randn(5,6,7,8) # test data

In [156]:
Xavg, Cov, _, _= array_normal(X0) # Fit an array-normal distribution

MLE for array normal converged in  4  steps


In [157]:
Xsample = anormal_sampling(Xavg,Cov) # sample from the modeled array-normal distribution

In [221]:
# Multilinear Regression with Array-normal distribution
# Ref:MULTILINEAR TENSOR REGRESSION FOR LONGITUDINAL RELATIONAL DATA:
# https://www.jstor.org/stable/43826417?seq=1#metadata_info_tab_contents
def anormal_regression(X,Y,tol=0.001):
    '''
    X: Tx?x?x..x? : {X_i}, i=1,..,T
    Y: Tx?x?x..x?: {Y_i}, i=1,...,T 
    N: dimension of the tensor X_i (or Y_i)
    J: ranks of the factor matrices
    return the model: Y_i = X_ix{U_1,..,U_N} + N(0,Cov)
    '''
    I = Y.shape
    J = X.shape
    T = X.shape[0]
    # Initialization of factor matrices
    U = [100*np.random.rand(I[n+1],J[n+1]) for n in range(N)] 
    #Note: Fix the initialization. Somehow, smaller values of initialization, results in slower convergence.
    # Understand this problem
    U.insert(0,np.eye(I[0])) 
    Y_apprx0 = multi_mode_dot(X,U,modes=[i for i in range(N+1)]) # current output approximation
    print("Shape of Y_apprx0: ",Y_apprx0.shape)
    # Initialize covariance matrices
    A = [100*np.random.rand(I[i+1],I[i+1]) for i in range(N)]
    Cov = [A_@A_.T for A_ in A]   
    A_inv = [None]*N
    for n in range(N):
        (A[n],A_inv[n]) = mat_root(Cov[n])
    A_inv.insert(0,np.identity(T))
    A.insert(0,np.identity(T))
    Cov.insert(0,np.identity(T))
    convergence = False
    for m in range(1000):
        # With the current estimation of Covariance (i.e A and A_inv) find the factor matrices U
        print("iteration: ",m)
        for reps in range(3):
            for n in range(N):
                U_ = U[:]
                U_[n+1] = np.eye(J[n+1])
                A_inv_ = A_inv[1:]
                A_inv_.pop(n)
                X_tmp = multi_mode_dot(X,U_,modes=[i for i in range(N+1)]) 
                modes=[i+1 for i in range(N)]
                modes.pop(n)
                X_tmp_rescaled = multi_mode_dot(X_tmp, A_inv_[1:],modes=modes)
                X_tmp_rescaled_n = unfold(X_tmp_rescaled,mode=n+1)
                Y_rescaled = multi_mode_dot(Y, A_inv_[1:],modes=modes)
                Y_rescaled_n = unfold(Y_rescaled,mode=n+1)
                U[n+1] = Y_rescaled_n@pinv(X_tmp_rescaled_n)
                if n<(N-1): # to make sure the values dont blow too much
                    U[n+1] = U[n+1]/np.linalg.norm(U[n+1])

        # With the current estimation of factor matrices find the Covariances
        Xe = Y-multi_mode_dot(X,U,modes=[i for i in range(N+1)]) # Error in approximation
        Xe = Xe-np.average(Xe,axis=0) # subtract the mean
        for reps in range(3):
            for n in range(N):
                A_inv_ = A_inv[:]
                A_ = A[:]
                Cov_ = Cov[:]
                A_[n+1] = np.eye(I[n+1])
                A_inv_[n+1] = np.eye(I[n+1])
                Xe_tmp = multi_mode_dot(Xe,A_inv_[1:],modes=[i+1 for i in range(N)])
                Xe_tmp_n = unfold(Xe_tmp,mode=n+1)
                c = T*np.prod(I[1:])/I[n+1]
                Cov[n+1] = (Xe_tmp_n@Xe_tmp_n.T)/c
                if n<(N-1):#to make sure the values dont blow too much
                    Cov[n+1] = Cov[n+1]/np.linalg.norm(Cov[n+1])
                (A[n+1],A_inv[n+1]) = mat_root(Cov[n+1])

        
        
        # Check convergence
        Y_apprx = multi_mode_dot(X,U,modes=[i for i in range(N+1)])#Curent approximation
        err_cov = np.linalg.norm(kron(Cov[1:])-kron(Cov_[1:]))
        err_reg = np.linalg.norm(Y_apprx-Y_apprx0)
        if  err_cov < tol and err_reg < tol:
            print("Regression converged at iteration: ",m)
            convergence = True
            break
        else:
            if (m+1)%1==0:
                print("Current error in approximation | U:", err_reg, " | Cov: ", err_cov)
            Y_apprx0 = 1*Y_apprx

    
    if convergence==False:
        print("Multilinear regression has not yet converged")
    print("Final error in approximation of Y = X x {U_1,..,U_N}:", np.linalg.norm(Y-Y_apprx)/np.linalg.norm(Y))
    return (U[1:],Cov[1:],A[1:], A_inv[1:])


In [222]:
X.shape

(10, 10, 10, 10)

In [223]:
U, Cov, A, A_inv = anormal_regression(X,Y,tol=0.001)

Shape of Y_apprx0:  (10, 20, 20, 20)
iteration:  0
Current error in approximation | U: 293341559489.3255  | Cov:  832.6220663330331
iteration:  1
Current error in approximation | U: 2337.407519836256  | Cov:  34.27515184579757
iteration:  2
Current error in approximation | U: 180.71850361907326  | Cov:  5.881777228993942
iteration:  3
Current error in approximation | U: 59.35263026243714  | Cov:  4.33087492395488
iteration:  4
Current error in approximation | U: 42.00369033197875  | Cov:  2.3583091481033804
iteration:  5
Current error in approximation | U: 27.62477150282561  | Cov:  1.2581003443353582
iteration:  6
Current error in approximation | U: 17.721480189876086  | Cov:  0.33146454356621396
iteration:  7
Current error in approximation | U: 11.042223026463693  | Cov:  0.10536270535460752
iteration:  8
Current error in approximation | U: 6.872670141019905  | Cov:  0.026374208541834386
iteration:  9
Current error in approximation | U: 4.240734039792945  | Cov:  0.006317095094210167

3